## A toy example of using the dgrid_components.py module in risktables, for both plots inside a jupyter notebook, and to run a web app from a jupyter notebook

In [1]:
from IPython.display import display
import dash
import sys,os
if  not os.path.abspath('./') in sys.path:
    sys.path.append(os.path.abspath('./'))
if  not os.path.abspath('../') in sys.path:
    sys.path.append(os.path.abspath('../'))

import datetime
import pandas as pd
import numpy as np

from risktables import dgrid_components as dgc
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import  init_notebook_mode, iplot
init_notebook_mode(connected=True)
import pathlib
SYSTEM_HOME = pathlib.Path.home()

### Generate a Pandas DataFrame ( ```df_pseudo```), with open, high, low and close data generated from a random walk process.

In [2]:
all_days=1000
end_date = datetime.datetime.now()
beg_date = end_date - datetime.timedelta(all_days)
date_series = pd.bdate_range(beg_date,end_date)
trade_dates = date_series.astype(str).str.replace('-','').astype(int)
n = len(trade_dates)
changes = np.random.lognormal(0,.15/256**.5,n-1)
initial = np.array([100.0])
closes = np.cumprod(np.append(initial,changes)).round(2)
open_ranges = np.random.lognormal(0,.3/256**.5,n)
opens = (closes * open_ranges).round(2)
low_ranges = np.random.lognormal(.1,.2/256**.5,n)
lows = np.array([min(x,y) for x,y in zip(opens,closes)]) - low_ranges
lows = lows.round(2)
high_ranges = np.random.lognormal(.1,.2/256**.5,n)
highs = np.array([max(x,y) for x,y in zip(opens,closes)]) + high_ranges
highs = highs.round(2)

volume_changes = np.random.lognormal(0,.50/256**.5,n-1)
initial_volume = np.array([1000000.0]) * np.random.lognormal(0,.15/256**.5)
volumes = np.cumprod(np.append(initial_volume,volume_changes)).round(0)


df_pseudo = dgc.make_df({'date':trade_dates,'open':opens,'high':highs,'low':lows,
                  'close':closes,'volume':volumes})


### Plot candlesticks using the data from ```df_pseudo```

In [4]:
fig1 = dgc.PlotlyCandles(df_pseudo.iloc[-50:],number_of_ticks_display=20,title='Random Walk')
iplot(fig1.get_figure())


In [5]:
import json
import plotly
f = json.dumps(fig1.get_figure(), cls=plotly.utils.PlotlyJSONEncoder)
open(f'{SYSTEM_HOME}/mydata5.json','w').write(f)


28094

### Create a small Plot.ly Dash web app using classes from the module ```dgrid_components.py```

In [6]:
no_border={
    'border': '1px solid #000',
    'grid-gap': '10px 10px',
}
df_save_list = []
logger = dgc.init_root_logger('logfile.log','INFO')

def dash_app_1():
    days_to_show = 50
    df_to_show = df_pseudo.iloc[-days_to_show:]
    
    dt1_comp = dgc.DashTableComponent(
        'dt1',df_to_show,None,title='Random Walk',
        editable_columns=['close'],logger = logger
    )
    def create_figure_from_df(input_list):
        if input_list is None or len(input_list)<1 or input_list[0] is None:
            return [None]
        dict_df = input_list[0]
        df = dgc.make_df(dict_df)
        df.close = df.close.astype(str).astype(float)
        p = dgc.PlotlyCandles(df,number_of_ticks_display=20,title='Random Walk').get_figure()
        return p
    
    gr1_comp = dgc.FigureComponent('f1',dt1_comp,create_figure_from_df,logger=logger)

    app = dash.Dash()
    comp_list = [dt1_comp,gr1_comp]
    html_list = [dgc.create_grid([c],num_columns=1) for c in comp_list]
    app.layout = dgc.html.Div(html_list)
    [c.callback(app) for c in comp_list]
    return app


### Launch the web app
##### To see the web app, open a browser to localhost:8500

In [ ]:
DASH_APP_TO_RUN = dash_app_1
app = DASH_APP_TO_RUN()
app.run_server(host='127.0.0.1',port=8500)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


2020-01-01 23:54:14,099 - werkzeug - INFO -  * Running on http://127.0.0.1:8500/ (Press CTRL+C to quit)
2020-01-01 23:54:30,408 - werkzeug - INFO - 127.0.0.1 - - [01/Jan/2020 23:54:30] "GET / HTTP/1.1" 200 -
2020-01-01 23:54:30,543 - werkzeug - INFO - 127.0.0.1 - - [01/Jan/2020 23:54:30] "GET /assets/custom.css?m=1564112807.0 HTTP/1.1" 200 -
2020-01-01 23:54:30,546 - werkzeug - INFO - 127.0.0.1 - - [01/Jan/2020 23:54:30] "GET /_dash-component-suites/dash_renderer/react@16.8.6.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-01 23:54:30,561 - werkzeug - INFO - 127.0.0.1 - - [01/Jan/2020 23:54:30] "GET /_dash-component-suites/dash_renderer/prop-types@15.7.2.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-01 23:54:30,567 - werkzeug - INFO - 127.0.0.1 - - [01/Jan/2020 23:54:30] "GET /_dash-component-suites/dash_html_components/dash_html_components.min.js?v=1.0.0&m=1574289295 HTTP/1.1" 200 -
2020-01-01 23:54:30,568 - werkzeug - INFO - 127.0.0.1 - - [01/Jan/2020 23:54:30] "GET /_dash-

### After stopping the above cell, see the layout "flattened"

In [7]:
l = dgc.flatten_layout(app)
for c in l:
    print('***********************************************')
    print(c)

***********************************************
DataTable(columns=[{'name': 'date', 'id': 'date', 'editable': False}, {'name': 'open', 'id': 'open', 'editable': False}, {'name': 'high', 'id': 'high', 'editable': False}, {'name': 'low', 'id': 'low', 'editable': False}, {'name': 'close', 'id': 'close', 'editable': True}, {'name': 'volume', 'id': 'volume', 'editable': False}], css=[{'selector': 'table', 'rule': 'width: 100%;'}], data=[{'date': 20190522.0, 'open': 123.0, 'high': 124.13, 'low': 121.43, 'close': 122.56, 'volume': 901192.0}, {'date': 20190523.0, 'open': 122.24, 'high': 126.36, 'low': 121.15, 'close': 125.24, 'volume': 891312.0}, {'date': 20190524.0, 'open': 129.57, 'high': 130.68, 'low': 125.55, 'close': 126.67, 'volume': 871264.0}, {'date': 20190527.0, 'open': 128.67, 'high': 129.79, 'low': 125.8, 'close': 126.9, 'volume': 868242.0}, {'date': 20190528.0, 'open': 124.17, 'high': 127.1, 'low': 123.07, 'close': 125.99, 'volume': 870383.0}, {'date': 20190529.0, 'open': 126.32, '